# Обратная задача кинематики для SCARA

In [ ]:
from matplotlib import pyplot as plt
from matplotlib import animation
import numpy as np
from IPython.display import HTML
%matplotlib notebook

In [ ]:
from kinematics import Vector, Quaternion, Transform
import graphics

Зададим длины звеньев:

Длина | Величина (мм)
------|--------------
$l_0$ | 220.2
$l_1$ | 200
$l_2$ | 250

In [ ]:
scara_l = [220.2, 200, 250]

Укажем диапазоны изменения обобщенных координат:

Координата | Минимальное | Максимальное
-----------|-------------|-------------
$q_0$      | -140 град   | 140 град
$q_1$      | -150 град   | 150 град
$q_2$      | -400 град   | 400 град
$q_3$      | 0 мм        | 180 мм

In [ ]:
scara_lim = [
    (-140, 140),
    (-150, 150),
    (-400, 400),
    (0, 180)
]

Возьмем готовое решение прямой задачи кинематики:

In [ ]:
def scara_chain(q, l):
    base = Transform.identity()
    column = base + Transform(
        Vector(0, 0, l[0]),
        Quaternion.from_angle_axis(q[0], Vector(0, 0, 1))
    )
    elbow = column + Transform(
        Vector(l[1], 0, 0),
        Quaternion.from_angle_axis(q[1], Vector(0, 0, 1))
    )
    tool = elbow + Transform(
        Vector(l[2], 0, 0),
        Quaternion.from_angle_axis(q[2], Vector(0, 0, 1))
    )
    flange = tool + Transform(
        Vector(0, 0, -q[3]),
        Quaternion.identity()
    )
    return [
        base,
        column,
        elbow,
        tool,
        flange
    ]

Для описания целевого положения будем использовать вектор и угол поворота вокруг вертикальной оси:

In [ ]:
class Target:
    def __init__(self, translation, angle):
        super(Target, self).__init__()
        self.translation = translation
        self.angle = angle
    
    def to_transform(self):
        return Transform(
            self.translation,
            Quaternion.from_angle_axis(
                self.angle,
                Vector(0, 0, 1)
            )
        )

## Самостоятельное задание

Самостоятельно решите обратную задачу кинематики:

In [ ]:
def wrap_from_to(value, s, e):
    r = e - s
    return value - (r * np.floor((value - s) / r))

def scara_ik(target, l):
    d = (target.translation.x ** 2 + target.translation.y ** 2) ** 0.5
    q0 = Vector(1, 0, 0).angle_to(
        Vector(target.translation.x, target.translation.y, 0),
        Vector(0, 0, 1)
    )
    q1 = np.pi / 2
    q2 = 0
    q3 = 100
    return (
        wrap_from_to(q0, -np.pi, np.pi),
        wrap_from_to(q1, -np.pi, np.pi),
        wrap_from_to(q2, -np.pi, np.pi),
        q3
    )

Зададим закон изменения целевого положения:

In [ ]:
def target(t, total):
    omega = t / total * np.pi * 2
    return Target(
        Vector(200, 0, 100) + 100 * Vector(np.cos(omega), np.sin(omega), 0),
        4 * omega
    )

In [ ]:
(x, y, z) = graphics.chain_to_points(
    scara_chain([0, 0, 0, 0], scara_l)
)
fig, ax = graphics.figure(600)
lines, = ax.plot(x, y, z, color="#000000")
rt, gt, bt = graphics.axis(ax, Transform.identity(), 1)
rf, gf, bf = graphics.axis(ax, Transform.identity(), 1)

total = 100

def animate(frame):
    t = target(frame, total)
    q = scara_ik(
        t,
        scara_l
    )
    chain = scara_chain(q, scara_l)
    (x, y, z) = graphics.chain_to_points(chain)
    lines.set_data_3d(x, y, z)
    global rt, gt, bt, rf, gf, bf
    rt.remove(); gt.remove(); bt.remove(); rf.remove(); gf.remove(); bf.remove()
    rt, gt, bt = graphics.axis(ax, t.to_transform(), 100)
    rf, gf, bf = graphics.axis(ax, chain[-1], 100)


animate(0)
fps = 25
scara_ani = animation.FuncAnimation(
    fig,
    animate,
    frames=total,
    interval=1000.0/fps
)

In [ ]:
HTML(scara_ani.to_jshtml())

In [ ]:
def scara_limited_ik(target, l):
    solution = scara_ik(target, l)
    if solution[0] < np.deg2rad(scara_lim[0][0]) or solution[0] > np.deg2rad(scara_lim[0][1]) or np.isnan(solution[0]):
        return None
    if solution[1] < np.deg2rad(scara_lim[1][0]) or solution[1] > np.deg2rad(scara_lim[1][1]) or np.isnan(solution[1]):
        return None
    if solution[2] < np.deg2rad(scara_lim[2][0]) or solution[2] > np.deg2rad(scara_lim[2][1]) or np.isnan(solution[2]):
        return None
    if solution[3] < scara_lim[3][0] or solution[3] > scara_lim[3][1] or np.isnan(solution[3]):
        return None
    return solution

Оценим рабочую зону:

In [ ]:
size = 600
step = 25

fig, ax = graphics.figure(size * 2)

px = []; py = []; pz = []

for x in np.arange(-size, size, step):
    for y in np.arange(-size, size, step):
        for z in np.arange(0, size, step):
            t = Target(Vector(x, y, z), 0)
            if scara_limited_ik(t, scara_l) != None:
                px += [x]
                py += [y]
                pz += [z]

ax.scatter(px, py, pz)
fig.show()

## Самостоятельное задание

- Для своего закона изменения целевого положения постройте графики изменения обобщенных координат;
- Оцените рабочую зону аналитически, постройте вид сверху в системе САПР